SwatGrains 2020 Data Analysis: Info.txt file - Properties of Jamming
===========

## 0.1. Load Data

Loads a info.txt file as the dataframe "data," adding rows for elastic moduli and percentage of rattlers, and disregarding rows which are not jammed or physically unreasonable.
Then uses linear regression to calculate $\phi_c$ for each trial, and generates a column of $\Delta\phi$ values in the dataframe.

In [ ]:
# READ DATA

import numpy as np
import scipy.optimize as sp
import matplotlib.pyplot as plt
import pandas as pd
import os
from scipy import stats

def linear_fit(xdata,slope,intercept):
    return xdata*slope+intercept

def read_data(filename):
    data = pd.read_csv(filename, sep=' ', usecols=range(19), dtype={'r_small':'str','a':'str'}) # read text file into dataframe
    # Add columns for elastic moduli
    #data = data.groupby(['r_small', 'percolation','num_pins']).mean().reset_index()
    bulkModulus = 0.25*(data['cxxxx'] + data['cyyyy']+2*data['cxxyy'])
    shearPrimeModulus = 0.25*(data['cxxxx'] + data['cyyyy'] - 2*data['cxxyy'])
    #data['rattlerfraction'] = data['rattler_count']/(data['num_total']-data['num_pins'])
    data['averageShear'] = 0.5*(data['cxyxy']+shearPrimeModulus).abs()
    data['bulkModulus'] = bulkModulus.abs()
    data['shearPrimeModulus'] = shearPrimeModulus.abs()
    data = data[(data.phi_norattlers<0.95) & (data.pressure>1e-8)]
    # remove bad data rows
    data = data[np.logical_or(data.percolation == 1, data.percolation == 3)]  # remove non-jammed cases
    data = data[data.bulkModulus > 0] # remove physically unreasonable elastic moduli
    data = data[np.logical_and(data.cxyxy>0,data.shearPrimeModulus>0)]
    return data

def read_new_data(filename):
    if filename[-8]=='7': # kind of roundabout: two different file formats mean we have to check which we have, right now all new data is from July
        data=pd.read_csv(filename, usecols=range(27), sep=' ', header=0, names=['phi', 'phi_norattlers', 'pressure', 'energy', 'percolation', 'rattler_count', 'cxxxx', 'cyyyy', 'cxyxy', 'cxxyy', 'cxxxy', 'cyyxy', 'axxxx', 'ayyyy', 'axyxy', 'axxyy', 'axxxy', 'ayyxy', 'space1', 'space2', 'geometry', 'seed', 'num_total', 'num_pins', 'r_small', 'a', 'geo_number'], dtype={'r_small':'str','a':'str'}).dropna(axis=1) # read bondInfo.txt
        bulkModulus = 0.25*(data['cxxxx'] + data['cyyyy']+2*data['cxxyy'])
        shearPrimeModulus = 0.25*(data['cxxxx'] + data['cyyyy'] - 2*data['cxxyy'])
        #data['rattlerfraction'] = data['rattler_count']/(data['num_total']-data['num_pins'])
        data['averageShear'] = 0.5*(data['cxyxy']+shearPrimeModulus).abs()
        data['bulkModulus'] = bulkModulus.abs()
        data['shearPrimeModulus'] = shearPrimeModulus.abs()
        data = data[(data.phi_norattlers<0.95) & (data.pressure>1e-8)]
        #affine moduli
        aBulkModulus = 0.25*(data['axxxx'] + data['ayyyy']+2*data['axxyy'])
        aShearPrimeModulus = 0.25*(data['axxxx'] + data['ayyyy'] - 2*data['axxyy'])
        data['aAverageShear'] = 0.5*(data['axyxy']+aShearPrimeModulus).abs()
        data['aBulkModulus'] = aBulkModulus.abs()
        data['aShearPrimeModulus'] = aShearPrimeModulus.abs()
        # remove bad data rows
        data = data[np.logical_or(data.percolation == 1, data.percolation == 3)]  # remove non-jammed cases
        data = data[data.bulkModulus > 0] # remove physically unreasonable elastic moduli
        data = data[np.logical_and(data.cxyxy>0,data.shearPrimeModulus>0)]
        return data
    else:
        return read_data(filename)


# parameters
filenames = {'squ64_07-12':'simulation_data/squ_Info07-12.txt', 'squ36_07-11':'simulation_data/squ_Info07-11.txt', 'tri36_07-11':'simulation_data/tri_Info07-11.txt', 'tri64_07-12':'simulation_data/tri_Info07-12.txt',
             'squ64_07-13':'simulation_data/squ_Info07-13.txt', 'non0_07-11':'simulation_data/non_Info07-11.txt', 'ran36_07-11':'simulation_data/ran_Info07-11.txt', 'non0_07-13':'simulation_data/non_Info07-13.txt',
             'non0_08-02':'simulation_data/ZeroPinData/non_Info08-02.txt', 'non0_06-20':'simulation_data/ZeroPinData/non_Info06-20.txt', 'squ16_08-05':'simulation_data/squ_InfoNp16_08-05.txt', 'squ64_08-05':'simulation_data/squ_InfoNp64_08-05.txt', 'tri64_08-08':'simulation_data/tri_InfoNp64_08-08.txt',
             'squ100_07-19':'simulation_data/squ_Info07-19.txt', 'tri100_07-19':'simulation_data/tri_Info07-19.txt', 'ran100_07-19':'simulation_data/ran_Info07-19.txt'
            }
files = ['non0_07-11', 'non0_07-13', 'squ36_07-11', 'squ64_07-12', 'tri36_07-11', 'tri64_07-12', 'ran36_07-11', 'squ100_07-19', 'tri100_07-19', 'ran100_07-19', 'non0_08-02', 'non0_06-20', 'squ16_08-05', 'squ64_08-05', 'tri64_08-08']
#files=['non0_08-02']
dataDict={file:read_new_data(filenames[file]) for file in files}

## 0.2. Writing a pressure text file for bondInfo_network.ipynb
To aid our work in the bondInfo_network.ipynb notebook, we need to output a text file containing pressure values for each jammed seed such that they are easily accessed by trial name. The trial name convention in bondInfo.txt is as follows:
$$\text{\{geometry\}-\{seed\}-\{particles\}-\{pins\}-\{small\}_\{radius\}-\{separation\}}$$
So we will output trial names also in this format.

In [ ]:
def pressure_file(dataset, file):
    filePath=file.split('/')
    filePath[-1]=filePath[-1].replace('Info','Pressures')
    minPressures=dataset.loc[dataset.groupby(['seed','r_small'], as_index=False)['pressure'].idxmin()]
    minPressures['trialName']=minPressures.apply(lambda x: '{}-{}-{}-{}-{}-{}'.format(*[str(x[key]) for key in ['geometry','seed','num_total','num_pins','r_small','a']]), axis=1)
    name_and_pressure=minPressures['pressure']
    name_and_pressure.index=minPressures['trialName']
    try:
        pressuresFile=pd.read_csv('/'.join(filePath), sep=' ', index_col=0)
    except:
        print('{} file is either empty or missing, generating new file'.format('/'.join(filePath)))
        pressuresFile=pd.DataFrame()
    pressuresFile['pressure']=name_and_pressure
    pressuresFile.to_csv('/'.join(filePath), sep=' ')
    return pressuresFile

for file in files:
    pressure_file(dataDict[file],filenames[file])

simulation_data/squ_Pressures07-19.txt file is either empty or missing, generating new file


simulation_data/tri_Pressures07-19.txt file is either empty or missing, generating new file


simulation_data/ran_Pressures07-19.txt file is either empty or missing, generating new file


## 0.3. Finding $\phi_c$ and $\Delta\phi$
Uses scipy.optimize.curve_fit() to find take a linear regression of $\phi$ against pressure. We expect pressure to go to zero at $\phi_c$, so the intercept of this trendline is the critical packing fraction. Then, create a $\Delta\phi$ column using the relation $\Delta\phi=\phi-\phi_c.$

In [ ]:
data = read_data('simulation_data/squInfo08-05.txt')
#deltaPhiFrame = pd.DataFrame(dtype='float64') # initialize a Series to add values to
phiCriticals_bySeed = []
phiCriticals_byGroup = []

# linear regression by seed
for trial in data.groupby(['num_pins', 'seed']): # split data by seed, pin number, and regress each trial
    [slope,intercept], covariance = sp.curve_fit(linear_fit,trial[1]['pressure'],trial[1]['phi']) # find critical phi value by taking the x intercept of pressure/phi data
    #deltaPhi = trial[1]['phi']-intercept # calculate delta phi
    #deltaPhiSeries = deltaPhiSeries.append(deltaPhi) # add delta phi values for this trial to the column

    phiCriticals_bySeed.append((trial[0][0], intercept)) # save critical phi values to plot separately

# linear regression for all data
for trial in data.groupby('num_pins'): # split data by pin number, and regress each trial
    [slope,intercept], covariance = sp.curve_fit(linear_fit,trial[1]['pressure'],trial[1]['phi']) # find critical phi value by taking the x intercept of pressure/phi data
    perror=np.sqrt(np.diag(covariance))

    phiCriticals_byGroup.append((trial[0], intercept, perror[1])) # save critical phi values to plot separately

#data['deltaPhi'] = deltaPhiSeries # add completed delta phi column to data set
#data = data[data.deltaPhi >= 0] # Only concerned with data after jamming

In [ ]:
seedFrame=pd.DataFrame(phiCriticals_bySeed)
groupFrame=pd.DataFrame(phiCriticals_byGroup)

groupFrame.columns=['num_pins', 'phi_c', 'err_phi_c']
groupFrame=groupFrame.set_index('num_pins')
seedFrame=seedFrame.groupby(0).agg({1:['mean','sem']})
seedFrame.columns=['phi_c', 'err_phi_c']
seedFrame.index.name='num_pins'

print('regressed by seed:\n', seedFrame)
print('\nregressed by pin number:\n', groupFrame)

plt.figure(figsize=(8,8))
for df in [seedFrame,groupFrame]:
    plt.errorbar(df.index, df['phi_c'], yerr=df['err_phi_c'],fmt='.', ms=10, alpha=0.6)
plt.xlabel('number of pins')
plt.ylabel('$\phi_c$')
plt.title('square lattice critical packing fraction vs number of pins')
plt.show()

regressed by seed:
              phi_c  err_phi_c
num_pins                     
16        0.839576   0.000173
36        0.833108   0.000205
64        0.825886   0.000237
100       0.808140   0.000306

regressed by pin number:
              phi_c  err_phi_c
num_pins                     
16        0.839523   0.000147
36        0.833031   0.000162
64        0.825801   0.000181
100       0.808065   0.000196


# 1. Pressure plots

## 1.1. Plot $\phi_c$ for each trial

Plots $\phi_c$ for each trial. Uncomment line 8 to save a .png to the plots subdirectory.

In [0]:
# A quick plot of the critical packing fraction correlation

if not os.path.exists('plots'): # check for/create subdirectory for images
    os.makedirs('plots')

plt.figure(figsize = (8,8)) # large, square plot
plt.scatter(*zip(*phiCriticals))
plt.xlabel(independentVariable)
plt.ylabel('$\phi_c$')
plt.title('$\phi_c$ vs. {}'.format(independentVariable))
#plt.savefig('plots/critical_packing_vs_{}.png'.format(independentVariable), dpi=300)
plt.show()

## 1.2 (Deprecated). Elastic Moduli K, G, and G' vs. Pressure (all values averaged)
This function is the bane of my existence and will be burned once it is sufficiently scavenged for parts.

In [ ]:
# PLOT DATA
#print('Shear Modulus:',data['cxyxy'].mean(),'+/-',data['cxyxy'].std())
#print('Bulk Modulus:',data['bulkModulus'].mean(),'+/-',data['bulkModulus'].std())
#print('Shear Prime Modulus:',data['shearPrimeModulus'].mean(),'+/-',data['shearPrimeModulus'].std())

def plot_data(dataset, xvar, yvar, variable, xrange=None, yrange=None, xlabel=False, ylabel=False, saveFigure=False, trendLine=True):
    # Plots specified x vs. y, with all seeds averaged for each phi, for each trial in the dataset on the same logarithmic axes.
    # NOTE: if trendline is used, give axis limits!
    # PARAMETERS:
    #     dataset       (grouped df) - as compiled by read_data()
    #     xvar/yvar     (str)        - header of x/y axis data column
    #     variable      (str)        - variable which is altered between trials
    #     xrange/yrange (tuple)      - minimum and maximum x/y limits
    #     xlabel/ylabel (str)        - labels for axes and title. If blank, xvar/yvar are used.
    #     saveFigure    (bool)       - if True, saves png to root folder.
    #     trendLine     (bool)       - if True, finds and plots a trendLine.

#    plt.figure(figsize=(8,8)) # large, square plot

    # checks for custom axis labels
    if not xlabel:
        xlabel = xvar
    if not ylabel:
        ylabel = yvar

    # plots and labels each trial
    colorCounter=0 # tick colors through the for loop - this makes sure trend lines are the same color as corresponding data
    for trial in dataset.groupby(variable):
        legendLabel = '{}: {}'.format(variable, trial[0])
# Option 1: Average values for each seed and plot with error bars
#        trialData=trial[1][trial[1].deltaPhi<=0.1] # isolate data right after jamming
#
#        # average all seeds for each phi value
#        varAverages=trialData.groupby('phi')[[xvar,yvar]].mean()
#        varStd=trialData.groupby('phi')[[xvar,yvar]].std()
#
#        # logarithmic axes, clip error bars at 0
#        plt.xscale("log", nonposx='clip')
#        plt.yscale("log", nonposy='clip')
#
#        # plot data
#        plt.errorbar(varAverages[xvar], varAverages[yvar], xerr=varStd[xvar],yerr=varStd[yvar], fmt='.', color='C{}'.format(colorCounter),ecolor=None, label=legendLabel, alpha=0.5)
#
# Option to plot un-averaged data
        plt.loglog(trial[1][xvar],trial[1][yvar],'.',alpha=0.6,label=legendLabel)

        # plot trend line for each trial
        [slope,intercept], covariance = sp.curve_fit(linear_fit,np.log10(trial[1][xvar]), np.log10(trial[1][yvar]))
        error=np.sqrt(np.diag(covariance))
        plotFailed=False # This is just a warning if used incorrectly, since our code is a little messy.
        try:
            #xValues=np.array(xrange)
            xValues=np.array(plt.xlim())
            yValues=xValues**slope*10**intercept
            plt.plot(xValues,yValues,color='C{}'.format(colorCounter))
        except TypeError:
            plotFailed=True
        colorCounter+=1

        if plotFailed:
            print('Trendline(s) omitted: make sure to specify axis limits.')

    # cosmetics
    title='{} vs. {}, slope: {} +/- {}'.format(ylabel,xlabel, round(slope,3), round(error[0],3))
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    #leg = plt.legend() # generate legend

    # custom axis limits, if specified
    if xrange != None:
        plt.xlim(xrange)
    if yrange != None:
        plt.ylim(yrange)

    # export figure as image
    if saveFigure:
        if not os.path.exists('plots'): # check for/create subdirectory for images
            os.makedirs('plots')
        imageName='plots/avg_{}_vs_{}.png'.format(yvar,xvar)
        plt.savefig(imageName, dpi=300)
        print('Figure saved as {}!'.format(imageName))

    # plots the figure
#    plt.show()

In [ ]:
files = ['non0_07-11', 'squ36_07-11', 'squ64_07-12', 'tri36_07-11', 'tri64_07-12', 'ran36_07-11', 'squ100_07-19', 'tri100_07-19', 'ran100_07-19']

for cutoffPressure in [0.1,2]:
    fig,axes=plt.subplots(1,3, figsize=(18,6))
    #fig, (ax1,ax2)=plt.subplots(2,1, figsize=(5,10))
    for colorCounter, file in enumerate(files):
        num_pins=int(file.split('_')[0][3:])
        geo=file.split('_')[0][:3]
        if geo=='non':
            marker='C0X'
        elif geo=='squ':
            marker='C1s'
        elif geo=='tri':
            marker='C2^'
        elif geo=='ran':
            marker='C3d'
        for ax, modulus, title in zip(axes, ['bulkModulus', 'cxyxy', 'shearPrimeModulus'], ['Bulk Modulus','Shear Modulus','Shear Prime Modulus']):
            [slope_lo,intercept_lo], covariance_lo = sp.curve_fit(linear_fit,np.log10(dataDict[file].loc[dataDict[file].pressure<cutoffPressure,'pressure']), np.log10(dataDict[file].loc[dataDict[file].pressure<cutoffPressure,modulus]))
            print(file, title, slope_lo, intercept_lo)
            error_lo=np.sqrt(np.diag(covariance_lo))
            ax.errorbar(num_pins/(num_pins+230), intercept_lo, yerr=error_lo[1], fmt=marker, capsize=3, alpha=0.6, label=geo)
            #ax.set_ylim(-0.01,0.2)
            ax.set_aspect(1./ax.get_data_ratio())
            ax.set_title(title, fontsize=16)
    handles, labels = plt.gca().get_legend_handles_labels()
    handles = [h[0] if isinstance(h, matplotlib.container.ErrorbarContainer) else h for h in handles]
    by_label = dict(zip(labels, handles))
    axes[2].legend(by_label.values(), by_label.keys(), loc = 'upper right')
    axes[1].set_xlabel('pin fraction')
    axes[0].set_ylabel('intercept')
    plt.suptitle('Affine Modulus Intercepts vs. Pin Fraction, Pressure < {cutoff}'.format(cutoff=cutoffPressure), y=0.94, fontsize=18)
    #plt.savefig('plots/modulusIntercepts/affine-{cutoff:.3f}.png'.format(cutoff=cutoffPressure), dpi=500)
    plt.show()

non0_07-11 Bulk Modulus 0.018032267399910665 1.9009686533083885
non0_07-11 Shear Modulus 0.4692133397991037 0.7366815685540449
non0_07-11 Shear Prime Modulus 0.5673990064984145 0.7767949730617987
squ36_07-11 Bulk Modulus 0.011244669892070625 1.864736893455281
squ36_07-11 Shear Modulus 0.6503043986927076 0.9275198187092655
squ36_07-11 Shear Prime Modulus 0.5213149124101278 0.6221931831176727
squ64_07-12 Bulk Modulus 0.07644722349142478 1.8806168393320044
squ64_07-12 Shear Modulus 0.49128029013815977 0.5677778116751628
squ64_07-12 Shear Prime Modulus 0.49652147985099837 0.5391643477638496
tri36_07-11 Bulk Modulus 0.01779968054844875 1.9339550634026739
tri36_07-11 Shear Modulus 0.554207652980957 0.8508958678511225
tri36_07-11 Shear Prime Modulus 0.5327850713661149 0.7148223642808854


tri64_07-12 Bulk Modulus 0.00853991541135879 1.8875294354772314
tri64_07-12 Shear Modulus 0.4450897045784542 0.594946153071459
tri64_07-12 Shear Prime Modulus 0.5348351812986102 0.6954648988623949
ran36_07-11 Bulk Modulus 0.004281853532553059 1.8520734655203264
ran36_07-11 Shear Modulus 0.5727387448588789 0.7690083827918766
ran36_07-11 Shear Prime Modulus 0.4306862089845289 0.5442957193629732
squ100_07-19 Bulk Modulus 0.10134294959883183 1.8689419756176908
squ100_07-19 Shear Modulus 0.434786831711568 0.6056993729187458
squ100_07-19 Shear Prime Modulus 0.5659614956376386 0.477749357311407
tri100_07-19 Bulk Modulus 0.13811403852578968 1.8548691925254466
tri100_07-19 Shear Modulus 0.3972426615520259 0.2136115041252904
tri100_07-19 Shear Prime Modulus 0.566944483443868 0.4415083855040107
ran100_07-19 Bulk Modulus 0.07976545041235716 1.7926212798755126
ran100_07-19 Shear Modulus 0.26664131456229645 0.20354630171282506


ran100_07-19 Shear Prime Modulus 0.666324783295291 0.6308076860455206


non0_07-11 Bulk Modulus 0.02717183489760422 1.9130893201749841
non0_07-11 Shear Modulus 0.5328710705127113 0.8322199837290836
non0_07-11 Shear Prime Modulus 0.5854618141947132 0.8078498508925259
squ36_07-11 Bulk Modulus 0.03521471049350429 1.9020801927179525
squ36_07-11 Shear Modulus 0.57001696268915 0.8067419366732257
squ36_07-11 Shear Prime Modulus 0.5619567250262876 0.68173222238491
squ64_07-12 Bulk Modulus 0.06984788330551561 1.8676645453458498
squ64_07-12 Shear Modulus 0.5277499932889006 0.630338736925358
squ64_07-12 Shear Prime Modulus 0.5298481504852932 0.5958909639413754


tri36_07-11 Bulk Modulus 0.03326436050813776 1.9565432517026635
tri36_07-11 Shear Modulus 0.4918508272654901 0.7381753288779676
tri36_07-11 Shear Prime Modulus 0.4922444309855385 0.6439036964846737
tri64_07-12 Bulk Modulus 0.04119365557503418 1.9343404511311273
tri64_07-12 Shear Modulus 0.4837847977590507 0.6616825757629851
tri64_07-12 Shear Prime Modulus 0.5235790348717804 0.6757616785292121
ran36_07-11 Bulk Modulus 0.028870370148554883 1.891828610342493
ran36_07-11 Shear Modulus 0.5334700343696998 0.7029147265752075
ran36_07-11 Shear Prime Modulus 0.5044168205659156 0.6550759082341624
squ100_07-19 Bulk Modulus 0.09574477461212312 1.8563482038991366
squ100_07-19 Shear Modulus 0.4545505089721201 0.6405194343975433


squ100_07-19 Shear Prime Modulus 0.5644258445738936 0.466192372052215
tri100_07-19 Bulk Modulus 0.1447204229572242 1.863244387050109
tri100_07-19 Shear Modulus 0.4863222061976169 0.3860779380222831
tri100_07-19 Shear Prime Modulus 0.5776945448377203 0.4485683556259116
ran100_07-19 Bulk Modulus 0.08734816715717597 1.8005384671532905
ran100_07-19 Shear Modulus 0.37460285431397455 0.3138678800770332
ran100_07-19 Shear Prime Modulus 0.5158293631159213 0.39584219910245333


In [ ]:
from os import listdir
import imageio
image_list=[]
for imageName in reversed(sorted(listdir('plots/presentation/bulkmodulus2'))):
    image_list.append(imageio.imread('plots/presentation/bulkmodulus2/'+imageName))
imageio.mimwrite('plots/presentation/bulkmodulus_animated.gif', image_list, duration = 0.3)

## 1.4. Elastic Moduli vs. Pressure
Combining data sets with similar parameters and lattice structures, curve fitting as one big data set with a high and low pressure fit for each modulus

In [ ]:
data=pd.concat([nonLo,nonHi])
fig=plt.figure(figsize=(12,12)) # Initialize large square for all three subplots
cutoffPressure=0.07
regressResult=[]


# Plot 1: Bulk Modulus
ax1 = fig.add_subplot(2,2,1) # initilalize axes
regressResult.append('Bulk Modulus:')
[slope,intercept], covariance = sp.curve_fit(linear_fit,np.log10(data.loc[data.pressure>cutoffPressure,'pressure']), np.log10(data.loc[data.pressure>cutoffPressure,'bulkModulus'])) # run linear regression
error=np.sqrt(np.diag(covariance))
plt.loglog(nonLo['pressure'],nonLo['bulkModulus'],'.',alpha=0.2, label = "6-30 data", color = 'C2')
[slope1,intercept1], covariance1 = sp.curve_fit(linear_fit,np.log10(data.loc[data.pressure<cutoffPressure,'pressure']), np.log10(data.loc[data.pressure<cutoffPressure,'bulkModulus'])) # run linear regression on data
error1=np.sqrt(np.diag(covariance1))
plt.loglog(nonHi['pressure'],nonHi['bulkModulus'],'.',alpha=0.2, label = "6-20 data", color = 'C4')
RMS=np.sqrt(sum((data.loc[data.pressure>cutoffPressure,'bulkModulus']-data.loc[data.pressure>cutoffPressure,'pressure']**slope*10**intercept)**2)/len(data[data.pressure>cutoffPressure]))
regressResult.append('RMS_hi: {R:.4f}'.format(R=RMS))
RMS1=np.sqrt(sum((data.loc[data.pressure<cutoffPressure,'bulkModulus']-data.loc[data.pressure<cutoffPressure,'pressure']**slope1*10**intercept1)**2)/len(data[data.pressure<cutoffPressure]))
regressResult.append('RMS_low: {R:.4f}'.format(R=RMS1))
ax1.set(aspect='equal',adjustable='datalim')
plt.title('bulk modulus vs. pressure')
plt.xlabel('pressure')
plt.ylabel('bulk modulus')
xValues=np.array(plt.xlim())
yValues=xValues**slope*10**intercept
plt.plot(xValues,yValues,color='C0', label='hi pressure, slope = {bm:.3f} +/- {err:.3f}'.format(bm=slope, err=error[0]))
xValues1=np.array(plt.xlim()) # get minimum and maximum x values
yValues1=xValues1**slope1*10**intercept1 # calculate corresponding line
plt.plot(xValues1,yValues1,color='C1',label='lo pressure, slope = {bm:.3f} +/- {err:.3f}'.format(bm=slope1, err=error1[0]))
plt.legend()

# Plot 2: Shear Modulus
ax2 = fig.add_subplot(2,2,2)
regressResult.append('Shear Modulus:')
[slope,intercept], covariance = sp.curve_fit(linear_fit,np.log10(data.loc[data.pressure>cutoffPressure,'pressure']), np.log10(data.loc[data.pressure>cutoffPressure,'cxyxy'])) # run linear regression on data
error=np.sqrt(np.diag(covariance))
plt.loglog(nonLo['pressure'],nonLo['cxyxy'],'.',alpha=0.2, label = "6-30 data", color = 'C2')
[slope1,intercept1], covariance1 = sp.curve_fit(linear_fit,np.log10(data.loc[data.pressure<cutoffPressure,'pressure']), np.log10(data.loc[data.pressure<cutoffPressure,'cxyxy'])) # run linear regression on data
error1=np.sqrt(np.diag(covariance1))
plt.loglog(nonHi['pressure'],nonHi['cxyxy'],'.',alpha=0.2, label = "6-20 data", color = 'C4')
RMS=np.sqrt(sum((data.loc[data.pressure>cutoffPressure,'cxyxy']-data.loc[data.pressure>cutoffPressure,'pressure']**slope*10**intercept)**2)/len(data[data.pressure>cutoffPressure]))
regressResult.append('RMS_hi: {R:.4f}'.format(R=RMS))
RMS1=np.sqrt(sum((data.loc[data.pressure<cutoffPressure,'cxyxy']-data.loc[data.pressure<cutoffPressure,'pressure']**slope1*10**intercept1)**2)/len(data[data.pressure<cutoffPressure]))
regressResult.append('RMS_low: {R:.4f}'.format(R=RMS1))
ax2.set(aspect='equal',adjustable='datalim')
plt.title('shear modulus vs. pressure')
plt.xlabel('pressure')
plt.ylabel('shear modulus')
xValues=np.array(plt.xlim())
yValues=xValues**slope*10**intercept
plt.plot(xValues,yValues,color='C0', label='hi pressure, slope = {sm:.3f} +/- {err:.3f}'.format(sm=slope, err=error[0]))
xValues1=np.array(plt.xlim()) # get minimum and maximum x values
yValues1=xValues1**slope1*10**intercept1 # calculate corresponding line
plt.plot(xValues1,yValues1,color='C1',label='lo pressure, slope = {sm:.3f} +/- {err:.3f}'.format(sm=slope1, err=error1[0]))
plt.legend()


# Plot 3: Shear Prime Modulus
ax3 = fig.add_subplot(2,2,3)
regressResult.append('Shear Prime Modulus:')
[slope,intercept], covariance = sp.curve_fit(linear_fit,np.log10(data.loc[data.pressure>cutoffPressure,'pressure']), np.log10(data.loc[data.pressure>cutoffPressure,'shearPrimeModulus'])) # run linear regression on data
error=np.sqrt(np.diag(covariance))
plt.loglog(nonLo['pressure'],nonLo['shearPrimeModulus'],'.',alpha=0.2, label = "6-30 data", color = 'C2')
[slope1,intercept1], covariance1 = sp.curve_fit(linear_fit,np.log10(data.loc[data.pressure<cutoffPressure,'pressure']), np.log10(data.loc[data.pressure<cutoffPressure,'shearPrimeModulus'])) # run linear regression on data
error1=np.sqrt(np.diag(covariance1))
plt.loglog(nonHi['pressure'],nonHi['shearPrimeModulus'],'.',alpha=0.2, label = "6-20 data", color = 'C4')
RMS=np.sqrt(sum((data.loc[data.pressure>cutoffPressure,'shearPrimeModulus']-data.loc[data.pressure>cutoffPressure,'pressure']**slope*10**intercept)**2)/len(data[data.pressure>cutoffPressure]))
regressResult.append('RMS_hi: {R:.4f}'.format(R=RMS))
RMS1=np.sqrt(sum((data.loc[data.pressure<cutoffPressure,'shearPrimeModulus']-data.loc[data.pressure<cutoffPressure,'pressure']**slope1*10**intercept1)**2)/len(data[data.pressure<cutoffPressure]))
regressResult.append('RMS_low: {R:.4f}'.format(R=RMS1))
ax2.set(aspect='equal',adjustable='datalim')
plt.title('shear prime modulus vs. pressure')
plt.xlabel('pressure')
plt.ylabel('shear prime modulus')
xValues=np.array(plt.xlim())
yValues=xValues**slope*10**intercept
plt.plot(xValues,yValues,color='C0', label='hi pressure, slope = {spm:.3f} +/- {err:.3f}'.format(spm=slope, err=error[0]))
xValues1=np.array(plt.xlim()) # get minimum and maximum x values
yValues1=xValues1**slope1*10**intercept1 # calculate corresponding line
plt.plot(xValues1,yValues1,color='C1', label='lo pressure, slope = {spm:.3f} +/- {err:.3f}'.format(spm=slope1, err=error1[0])) # plot line by start and end points
plt.legend()

fig.suptitle('920 particle no pin 6-30 and 6-20', x=0.5,y=0.95, fontsize=16) # title of entire figure
plt.figtext(0.97,0.25,'\n'.join(regressResult), ha='right', fontsize=12)
plt.tight_layout()
plt.subplots_adjust(top=0.9)
#plt.savefig('plots/elasticModuli-920-noPin-separated.png', dpi=300)
plt.show()

## 1.6. Elastic Moduli vs. $\Delta$z for Different Pin Densities, Same Lattice

In [ ]:
def merge_data(file, data):
    # gets corresponding pressures.txt file and adds delta z columns to each trial.
    filePath=file.split('/')
    filePath[-1]=filePath[-1].replace('Info','Pressures')
    trialPressures=pd.read_csv('/'.join(filePath), sep=' ', index_col=0, usecols=['trialName','delta_z'])

    minPressures=data.loc[data.groupby(['seed','r_small'], as_index=False)['pressure'].idxmin()]

    minPressures.index=minPressures.apply(lambda x: '{}-{}-{}-{}-{}-{}'.format(*[str(x[key]) for key in ['geometry','seed','num_total','num_pins','r_small','a']]), axis=1)
    minPressures.index.name='trialName'
    minPressures=minPressures.merge(trialPressures, on='trialName',how='left')
    minPressures=minPressures[minPressures.delta_z>0]
    return minPressures

#minPressureDict={file:merge_data(filenames[file],dataDict[file]) for file in files}
minPressureDict={file:dataDict[file] for file in files}
fig=plt.figure(figsize=(18,6)) # Initialize large square for all three subplots
cutoffPressure=0.1

regressResult=[]
# Plot 1: Bulk Modulus
regressResult.append('Bulk Modulus:')
ax1 = fig.add_subplot(1,3,1) # initilalize axes
ax1.set_xlim(6.8e-5,1.38)
# linear regression
for colorCounter, file in enumerate(files):
    minPressures=minPressureDict[file]
    pinDensity=minPressures['num_pins'].iloc[0]/(minPressures['num_total'].iloc[0]-minPressures['num_pins'].iloc[0])
    plt.loglog(minPressures['pressure'],minPressures['bulkModulus'], '.', alpha=0.3,color='C{}'.format(colorCounter))
    [avgSlope,avgIntercept], avgCovariance = sp.curve_fit(linear_fit,np.log10(minPressures['pressure']), np.log10(minPressures['bulkModulus']))
    avgError=np.sqrt(np.diag(avgCovariance))

    [slope_hi,intercept_hi], covariance_hi = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'pressure']), np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'bulkModulus']))
    error_hi=np.sqrt(np.diag(covariance_hi)) # find error on parameters
    [slope_lo,intercept_lo], covariance_lo = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'pressure']), np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'bulkModulus']))
    error_lo=np.sqrt(np.diag(covariance_lo))
    RMS_hi=np.sqrt(sum((minPressures.loc[minPressures.pressure>cutoffPressure,'bulkModulus']-minPressures.loc[minPressures.pressure>cutoffPressure,'pressure']**slope_hi*10**intercept_hi)**2)/len(minPressures[minPressures.pressure>cutoffPressure]))
    RMS_lo=np.sqrt(sum((minPressures.loc[minPressures.pressure<cutoffPressure,'bulkModulus']-minPressures.loc[minPressures.pressure<cutoffPressure,'pressure']**slope_lo*10**intercept_lo)**2)/len(minPressures[minPressures.pressure<cutoffPressure]))

    #regressResult.append('RMS_low: {R:.4f}'.format(R=RMS1))

    xValues=np.array(plt.xlim())
    yValues=xValues**slope_lo*10**intercept_lo
    plt.plot(xValues,yValues,color='C{}'.format(colorCounter), label='slope = {bm:.3f}+/-{err:.3f}'.format(bm=slope_lo,err=error_lo[0]))
# cosmetics
ax1.set(aspect='equal',adjustable='datalim')
plt.title('bulk modulus vs. pressure')
plt.xlabel('pressure')
plt.ylabel('bulk modulus')
plt.legend()

# Plot 2: Shear Modulus
regressResult.append('Shear Modulus:')
ax2 = fig.add_subplot(1,3,2) # initilalize axes
ax2.set_xlim(6.8e-5,1.38)
# linear regression
for colorCounter, file in enumerate(files):
    minPressures=minPressureDict[file]
    pinDensity=minPressures['num_pins'].iloc[0]/(minPressures['num_total'].iloc[0]-minPressures['num_pins'].iloc[0])
    plt.loglog(minPressures['pressure'],minPressures['cxyxy'], '.', alpha=0.3,color='C{}'.format(colorCounter))
    [avgSlope,avgIntercept], avgCovariance = sp.curve_fit(linear_fit,np.log10(minPressures['pressure']), np.log10(minPressures['cxyxy']))
    avgError=np.sqrt(np.diag(avgCovariance))

    [slope_hi,intercept_hi], covariance_hi = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'pressure']), np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'cxyxy']))
    error_hi=np.sqrt(np.diag(covariance_hi)) # find error on parameters
    [slope_lo,intercept_lo], covariance_lo = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'pressure']), np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'cxyxy']))
    error_lo=np.sqrt(np.diag(covariance_lo))
    print(file, intercept_lo)
    RMS_hi=np.sqrt(sum((minPressures.loc[minPressures.pressure>cutoffPressure,'cxyxy']-minPressures.loc[minPressures.pressure>cutoffPressure,'pressure']**slope_hi*10**intercept_hi)**2)/len(minPressures[minPressures.pressure>cutoffPressure]))
    RMS_lo=np.sqrt(sum((minPressures.loc[minPressures.pressure<cutoffPressure,'cxyxy']-minPressures.loc[minPressures.pressure<cutoffPressure,'pressure']**slope_lo*10**intercept_lo)**2)/len(minPressures[minPressures.pressure<cutoffPressure]))

    #regressResult.append('RMS_low: {R:.4f}'.format(R=RMS1))

    xValues=np.array(plt.xlim())
    yValues=xValues**slope_lo*10**intercept_lo
    plt.plot(xValues,yValues,color='C{}'.format(colorCounter), label='slope = {bm:.3f}+/-{err:.3f}'.format(bm=slope_lo,err=error_lo[0]))
# cosmetics
ax2.set(aspect='equal',adjustable='datalim')
plt.title('shear modulus vs. pressure')
plt.xlabel('pressure')
plt.ylabel('shear modulus')
plt.legend()

# Plot 3: Shear Prime Modulus
regressResult.append('Shear Prime Modulus:')
ax3 = fig.add_subplot(1,3,3) # initilalize axes
ax3.set_xlim(6.8e-5,1.38)
# linear regression
for colorCounter, file in enumerate(files):
    minPressures=minPressureDict[file]
    pinDensity=minPressures['num_pins'].iloc[0]/(minPressures['num_total'].iloc[0]-minPressures['num_pins'].iloc[0])
    plt.loglog(minPressures['pressure'],minPressures['shearPrimeModulus'], '.', alpha=0.3,color='C{}'.format(colorCounter))
    [avgSlope,avgIntercept], avgCovariance = sp.curve_fit(linear_fit,np.log10(minPressures['pressure']), np.log10(minPressures['shearPrimeModulus']))
    avgError=np.sqrt(np.diag(avgCovariance))

    [slope_hi,intercept_hi], covariance_hi = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'pressure']), np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'shearPrimeModulus']))
    error_hi=np.sqrt(np.diag(covariance_hi)) # find error on parameters
    [slope_lo,intercept_lo], covariance_lo = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'pressure']), np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'shearPrimeModulus']))
    error_lo=np.sqrt(np.diag(covariance_lo))
    RMS_hi=np.sqrt(sum((minPressures.loc[minPressures.pressure>cutoffPressure,'shearPrimeModulus']-minPressures.loc[minPressures.pressure>cutoffPressure,'pressure']**slope_hi*10**intercept_hi)**2)/len(minPressures[minPressures.pressure>cutoffPressure]))
    RMS_lo=np.sqrt(sum((minPressures.loc[minPressures.pressure<cutoffPressure,'shearPrimeModulus']-minPressures.loc[minPressures.pressure<cutoffPressure,'pressure']**slope_lo*10**intercept_lo)**2)/len(minPressures[minPressures.pressure<cutoffPressure]))

    #regressResult.append('RMS_low: {R:.4f}'.format(R=RMS1))

    xValues=np.array(plt.xlim())
    yValues=xValues**slope_lo*10**intercept_lo
    plt.plot(xValues,yValues,color='C{}'.format(colorCounter), label='slope = {bm:.3f}+/-{err:.3f}'.format(bm=slope_lo,err=error_lo[0]))
# cosmetics
ax3.set(aspect='equal',adjustable='datalim')
plt.title('shear prime modulus vs. pressure')
plt.xlabel('pressure')
plt.ylabel('G\' modulus')
plt.legend()
plt.subplots_adjust(top=0.85)

#plt.figtext(0.8,0.25,'\n'.join(regressResult), ha='right', fontsize=12)
plt.suptitle('Elastic Moduli vs. Pressure for July Data',x=0.5,y=0.93, fontsize = 16)

#plt.savefig('plots/presentation/elastic_moduli-non.png', dpi=300)

non0_07-11 0.7366815685540449
squ36_07-11 0.9275198187092655
squ64_07-12 0.5677778116751628
tri36_07-11 0.8508958678511225
tri64_07-12 0.594946153071459
ran36_07-11 0.7690083827918766
squ100_07-19 0.6056993729187458


tri100_07-19 0.2136115041252904
ran100_07-19 0.20354630171282506


Text(0.5, 0.93, 'Elastic Moduli vs. Pressure for July Data')

In [ ]:
def merge_data(file, data):
    # gets corresponding pressures.txt file and adds delta z columns to each trial.
    filePath=file.split('/')
    filePath[-1]=filePath[-1].replace('Info','Pressures')
    trialPressures=pd.read_csv('/'.join(filePath), sep=' ', index_col=0, usecols=['trialName','delta_z'])

    minPressures=data.loc[data.groupby(['seed','r_small'], as_index=False)['pressure'].idxmin()]

    minPressures.index=minPressures.apply(lambda x: '{}-{}-{}-{}-{}-{}'.format(*[str(x[key]) for key in ['geometry','seed','num_total','num_pins','r_small','a']]), axis=1)
    minPressures.index.name='trialName'
    minPressures=minPressures.merge(trialPressures, on='trialName',how='left')
    minPressures=minPressures[minPressures.delta_z>0]
    return minPressures

files=['non0_07-11','squ36_07-11','squ64_07-12']

minPressureDict={file:merge_data(filenames[file],dataDict[file]) for file in files}
fig=plt.figure(figsize=(18,6)) # Initialize large square for all three subplots
cutoffPressure=0.07

regressResult=[]

# Plot 1: Bulk Modulus
regressResult.append('Bulk Modulus:')
ax1 = fig.add_subplot(1,3,1) # initilalize axes
ax1.set_xlim(6.8e-5,1.38)
# linear regression
for colorCounter, file in enumerate(files):
    minPressures=minPressureDict[file]
    pinDensity=minPressures['num_pins'].iloc[0]/(minPressures['num_total'].iloc[0]-minPressures['num_pins'].iloc[0])
    plt.loglog(minPressures['delta_z'],minPressures['aBulkModulus'], '.', alpha=0.15,color='C{}'.format(colorCounter))
    [avgSlope,avgIntercept], avgCovariance = sp.curve_fit(linear_fit,np.log10(minPressures['delta_z']), np.log10(minPressures['aBulkModulus']))
    avgError=np.sqrt(np.diag(avgCovariance))

    [slope_hi,intercept_hi], covariance_hi = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'delta_z']), np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'aBulkModulus']))
    error_hi=np.sqrt(np.diag(covariance_hi)) # find error on parameters
    [slope_lo,intercept_lo], covariance_lo = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'delta_z']), np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'aBulkModulus']))
    error_lo=np.sqrt(np.diag(covariance_lo))
    RMS_hi=np.sqrt(sum((minPressures.loc[minPressures.pressure>cutoffPressure,'aBulkModulus']-minPressures.loc[minPressures.pressure>cutoffPressure,'delta_z']**slope_hi*10**intercept_hi)**2)/len(minPressures[minPressures.pressure>cutoffPressure]))
    RMS_lo=np.sqrt(sum((minPressures.loc[minPressures.pressure<cutoffPressure,'aBulkModulus']-minPressures.loc[minPressures.pressure<cutoffPressure,'delta_z']**slope_lo*10**intercept_lo)**2)/len(minPressures[minPressures.pressure<cutoffPressure]))

    #regressResult.append('RMS_low: {R:.4f}'.format(R=RMS1))

    xValues=np.array(plt.xlim())
    yValues=xValues**slope_lo*10**intercept_lo
    plt.plot(xValues,yValues,color='C{}'.format(colorCounter), label='{file} n_f: {n_f:.3f}, slope = {bm:.3f}+/-{err:.3f}'.format(file=file, n_f=pinDensity,bm=slope_lo,err=error_lo[0]))
# cosmetics
#ax1.set(aspect='equal',adjustable='datalim')
plt.title('affine bulk modulus vs. $\Delta Z$')
plt.xlabel('$\Delta$Z')
plt.ylabel('affine bulk modulus')
plt.legend()

# Plot 2: Shear Modulus
regressResult.append('Shear Modulus:')
ax2 = fig.add_subplot(1,3,2) # initilalize axes
ax2.set_xlim(6.8e-5,1.38)
# linear regression
for colorCounter, file in enumerate(files):
    minPressures=minPressureDict[file]
    pinDensity=minPressures['num_pins'].iloc[0]/(minPressures['num_total'].iloc[0]-minPressures['num_pins'].iloc[0])
    plt.loglog(minPressures['delta_z'],minPressures['axyxy'], '.', alpha=0.15,color='C{}'.format(colorCounter))
    [avgSlope,avgIntercept], avgCovariance = sp.curve_fit(linear_fit,np.log10(minPressures['delta_z']), np.log10(minPressures['axyxy']))
    avgError=np.sqrt(np.diag(avgCovariance))

    [slope_hi,intercept_hi], covariance_hi = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'delta_z']), np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'axyxy']))
    error_hi=np.sqrt(np.diag(covariance_hi)) # find error on parameters
    [slope_lo,intercept_lo], covariance_lo = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'delta_z']), np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'axyxy']))
    error_lo=np.sqrt(np.diag(covariance_lo))
    RMS_hi=np.sqrt(sum((minPressures.loc[minPressures.pressure>cutoffPressure,'axyxy']-minPressures.loc[minPressures.pressure>cutoffPressure,'delta_z']**slope_hi*10**intercept_hi)**2)/len(minPressures[minPressures.pressure>cutoffPressure]))
    RMS_lo=np.sqrt(sum((minPressures.loc[minPressures.pressure<cutoffPressure,'axyxy']-minPressures.loc[minPressures.pressure<cutoffPressure,'delta_z']**slope_lo*10**intercept_lo)**2)/len(minPressures[minPressures.pressure<cutoffPressure]))

    #regressResult.append('RMS_low: {R:.4f}'.format(R=RMS1))

    xValues=np.array(plt.xlim())
    yValues=xValues**slope_lo*10**intercept_lo
    plt.plot(xValues,yValues,color='C{}'.format(colorCounter), label='{file}, n_f: {n_f:.3f}, slope = {bm:.3f}+/-{err:.3f}'.format(file=file, n_f=pinDensity,bm=slope_lo,err=error_lo[0]))
# cosmetics
#ax1.set(aspect='equal',adjustable='datalim')
plt.title('affine shear modulus vs. $\Delta Z$')
plt.xlabel('$\Delta$Z')
plt.ylabel('affine shear modulus')
plt.legend()

# Plot 3: Shear Prime Modulus
regressResult.append('Shear Prime Modulus:')
ax3 = fig.add_subplot(1,3,3) # initilalize axes
ax3.set_xlim(6.8e-5,1.38)
# linear regression
for colorCounter, file in enumerate(files):
    minPressures=minPressureDict[file]
    pinDensity=minPressures['num_pins'].iloc[0]/(minPressures['num_total'].iloc[0]-minPressures['num_pins'].iloc[0])
    plt.loglog(minPressures['delta_z'],minPressures['aShearPrimeModulus'], '.', alpha=0.15,color='C{}'.format(colorCounter))
    [avgSlope,avgIntercept], avgCovariance = sp.curve_fit(linear_fit,np.log10(minPressures['delta_z']), np.log10(minPressures['aShearPrimeModulus']))
    avgError=np.sqrt(np.diag(avgCovariance))

    [slope_hi,intercept_hi], covariance_hi = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'delta_z']), np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'aShearPrimeModulus']))
    error_hi=np.sqrt(np.diag(covariance_hi)) # find error on parameters
    [slope_lo,intercept_lo], covariance_lo = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'delta_z']), np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'aShearPrimeModulus']))
    error_lo=np.sqrt(np.diag(covariance_lo))
    RMS_hi=np.sqrt(sum((minPressures.loc[minPressures.pressure>cutoffPressure,'aShearPrimeModulus']-minPressures.loc[minPressures.pressure>cutoffPressure,'delta_z']**slope_hi*10**intercept_hi)**2)/len(minPressures[minPressures.pressure>cutoffPressure]))
    RMS_lo=np.sqrt(sum((minPressures.loc[minPressures.pressure<cutoffPressure,'aShearPrimeModulus']-minPressures.loc[minPressures.pressure<cutoffPressure,'delta_z']**slope_lo*10**intercept_lo)**2)/len(minPressures[minPressures.pressure<cutoffPressure]))

    #regressResult.append('RMS_low: {R:.4f}'.format(R=RMS1))

    xValues=np.array(plt.xlim())
    yValues=xValues**slope_lo*10**intercept_lo
    plt.plot(xValues,yValues,color='C{}'.format(colorCounter), label='{file} n_f: {n_f:.3f}, slope = {bm:.3f}+/-{err:.3f}'.format(file=file, n_f=pinDensity,bm=slope_lo,err=error_lo[0]))
    plt.axvline(x=cutoffPressure)
# cosmetics
#ax1.set(aspect='equal',adjustable='datalim')
plt.title('affine shear prime modulus vs. $\Delta Z$')
plt.xlabel('$\Delta$Z')
plt.ylabel('affine G\' modulus')
plt.legend()

plt.subplots_adjust(top=0.9)
#plt.figtext(0.8,0.25,'\n'.join(regressResult), ha='right', fontsize=12)
plt.suptitle('Affine Elastic Moduli vs. $\Delta Z$ for non/tri lattice, July 2020, fitting p<{}'.format(cutoffPressure),x=0.5,y=0.93, fontsize = 16)

#plt.savefig('plots/affine_elasticmoduli-deltaz-tri-july2020.png', dpi=300)

Text(0.5, 0.93, 'Affine Elastic Moduli vs. $\\Delta Z$ for non/tri lattice, July 2020, fitting p<0.07')

In [ ]:
def merge_data(file, data):
    # gets corresponding pressures.txt file and adds delta z columns to each trial.
    filePath=file.split('/')
    filePath[-1]=filePath[-1].replace('Info','Pressures')
    trialPressures=pd.read_csv('/'.join(filePath), sep=' ', index_col=0, usecols=['trialName','delta_z'])

    minPressures=data.loc[data.groupby(['seed','r_small'], as_index=False)['pressure'].idxmin()]

    minPressures.index=minPressures.apply(lambda x: '{}-{}-{}-{}-{}-{}'.format(*[str(x[key]) for key in ['geometry','seed','num_total','num_pins','r_small','a']]), axis=1)
    minPressures.index.name='trialName'
    minPressures=minPressures.merge(trialPressures, on='trialName',how='left')
    minPressures=minPressures[minPressures.delta_z>0]
    return minPressures

files=['non0_07-11','squ36_07-11','squ64_07-12']

minPressureDict={file:merge_data(filenames[file],dataDict[file]) for file in files}
fig=plt.figure(figsize=(18,6)) # Initialize large square for all three subplots
cutoffPressure=0.1

regressResult=[]

# Plot 1: Bulk Modulus
regressResult.append('Bulk Modulus:')
ax1 = fig.add_subplot(1,3,1) # initilalize axes
ax1.set_xlim(6.8e-5,1.38)
# linear regression
for colorCounter, file in enumerate(files):
    minPressures=minPressureDict[file]
    pinDensity=minPressures['num_pins'].iloc[0]/(minPressures['num_total'].iloc[0]-minPressures['num_pins'].iloc[0])
    plt.loglog(minPressures['pressure'],minPressures['aBulkModulus'], '.', alpha=0.3,color='C{}'.format(colorCounter))
    [avgSlope,avgIntercept], avgCovariance = sp.curve_fit(linear_fit,np.log10(minPressures['pressure']), np.log10(minPressures['aBulkModulus']))
    avgError=np.sqrt(np.diag(avgCovariance))

    [slope_hi,intercept_hi], covariance_hi = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'pressure']), np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'aBulkModulus']))
    error_hi=np.sqrt(np.diag(covariance_hi)) # find error on parameters
    [slope_lo,intercept_lo], covariance_lo = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'pressure']), np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'aBulkModulus']))
    error_lo=np.sqrt(np.diag(covariance_lo))
    RMS_hi=np.sqrt(sum((minPressures.loc[minPressures.pressure>cutoffPressure,'aBulkModulus']-minPressures.loc[minPressures.pressure>cutoffPressure,'pressure']**slope_hi*10**intercept_hi)**2)/len(minPressures[minPressures.pressure>cutoffPressure]))
    RMS_lo=np.sqrt(sum((minPressures.loc[minPressures.pressure<cutoffPressure,'aBulkModulus']-minPressures.loc[minPressures.pressure<cutoffPressure,'pressure']**slope_lo*10**intercept_lo)**2)/len(minPressures[minPressures.pressure<cutoffPressure]))

    #regressResult.append('RMS_low: {R:.4f}'.format(R=RMS1))

    xValues=np.array(plt.xlim())
    yValues=xValues**slope_lo*10**intercept_lo
    plt.plot(xValues,yValues,color='C{}'.format(colorCounter), label='{file} n_f: {n_f:.3f}, slope = {bm:.3f}+/-{err:.3f}'.format(file=file, n_f=pinDensity,bm=slope_lo,err=error_lo[0]))
# cosmetics
#ax1.set(aspect='equal',adjustable='datalim')
plt.title('affine bulk modulus vs. pressure')
plt.xlabel('pressure')
plt.ylabel('affine bulk modulus')
plt.legend()

# Plot 2: Shear Modulus
regressResult.append('Shear Modulus:')
ax2 = fig.add_subplot(1,3,2) # initilalize axes
ax2.set_xlim(6.8e-5,1.38)
# linear regression
for colorCounter, file in enumerate(files):
    minPressures=minPressureDict[file]
    pinDensity=minPressures['num_pins'].iloc[0]/(minPressures['num_total'].iloc[0]-minPressures['num_pins'].iloc[0])
    plt.loglog(minPressures['pressure'],minPressures['axyxy'], '.', alpha=0.3,color='C{}'.format(colorCounter))
    [avgSlope,avgIntercept], avgCovariance = sp.curve_fit(linear_fit,np.log10(minPressures['pressure']), np.log10(minPressures['axyxy']))
    avgError=np.sqrt(np.diag(avgCovariance))

    [slope_hi,intercept_hi], covariance_hi = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'pressure']), np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'axyxy']))
    error_hi=np.sqrt(np.diag(covariance_hi)) # find error on parameters
    [slope_lo,intercept_lo], covariance_lo = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'pressure']), np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'axyxy']))
    error_lo=np.sqrt(np.diag(covariance_lo))
    RMS_hi=np.sqrt(sum((minPressures.loc[minPressures.pressure>cutoffPressure,'axyxy']-minPressures.loc[minPressures.pressure>cutoffPressure,'pressure']**slope_hi*10**intercept_hi)**2)/len(minPressures[minPressures.pressure>cutoffPressure]))
    RMS_lo=np.sqrt(sum((minPressures.loc[minPressures.pressure<cutoffPressure,'axyxy']-minPressures.loc[minPressures.pressure<cutoffPressure,'pressure']**slope_lo*10**intercept_lo)**2)/len(minPressures[minPressures.pressure<cutoffPressure]))

    #regressResult.append('RMS_low: {R:.4f}'.format(R=RMS1))

    xValues=np.array(plt.xlim())
    yValues=xValues**slope_lo*10**intercept_lo
    plt.plot(xValues,yValues,color='C{}'.format(colorCounter), label='{file}, n_f: {n_f:.3f}, slope = {bm:.3f}+/-{err:.3f}'.format(file=file, n_f=pinDensity,bm=slope_lo,err=error_lo[0]))
# cosmetics
#ax1.set(aspect='equal',adjustable='datalim')
plt.title('affine shear modulus vs. pressure')
plt.xlabel('pressure')
plt.ylabel('affine shear modulus')
plt.legend()

# Plot 3: Shear Prime Modulus
regressResult.append('Shear Prime Modulus:')
ax3 = fig.add_subplot(1,3,3) # initilalize axes
ax3.set_xlim(6.8e-5,1.38)
# linear regression
for colorCounter, file in enumerate(files):
    minPressures=minPressureDict[file]
    pinDensity=minPressures['num_pins'].iloc[0]/(minPressures['num_total'].iloc[0]-minPressures['num_pins'].iloc[0])
    plt.loglog(minPressures['pressure'],minPressures['aShearPrimeModulus'], '.', alpha=0.3,color='C{}'.format(colorCounter))
    [avgSlope,avgIntercept], avgCovariance = sp.curve_fit(linear_fit,np.log10(minPressures['pressure']), np.log10(minPressures['aShearPrimeModulus']))
    avgError=np.sqrt(np.diag(avgCovariance))

    [slope_hi,intercept_hi], covariance_hi = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'pressure']), np.log10(minPressures.loc[minPressures.pressure>cutoffPressure,'aShearPrimeModulus']))
    error_hi=np.sqrt(np.diag(covariance_hi)) # find error on parameters
    [slope_lo,intercept_lo], covariance_lo = sp.curve_fit(linear_fit,np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'pressure']), np.log10(minPressures.loc[minPressures.pressure<cutoffPressure,'aShearPrimeModulus']))
    error_lo=np.sqrt(np.diag(covariance_lo))
    RMS_hi=np.sqrt(sum((minPressures.loc[minPressures.pressure>cutoffPressure,'aShearPrimeModulus']-minPressures.loc[minPressures.pressure>cutoffPressure,'pressure']**slope_hi*10**intercept_hi)**2)/len(minPressures[minPressures.pressure>cutoffPressure]))
    RMS_lo=np.sqrt(sum((minPressures.loc[minPressures.pressure<cutoffPressure,'aShearPrimeModulus']-minPressures.loc[minPressures.pressure<cutoffPressure,'pressure']**slope_lo*10**intercept_lo)**2)/len(minPressures[minPressures.pressure<cutoffPressure]))

    #regressResult.append('RMS_low: {R:.4f}'.format(R=RMS1))

    xValues=np.array(plt.xlim())
    yValues=xValues**slope_lo*10**intercept_lo
    plt.plot(xValues,yValues,color='C{}'.format(colorCounter), label='{file} n_f: {n_f:.3f}, slope = {bm:.3f}+/-{err:.3f}'.format(file=file, n_f=pinDensity,bm=slope_lo,err=error_lo[0]))
# cosmetics
#ax1.set(aspect='equal',adjustable='datalim')
for axis in [ax1,ax2,ax3]:
    axis.axvspan(cutoffPressure,1.38, alpha=0.2, color='gray')
plt.title('affine shear prime modulus vs. pressure')
plt.xlabel('pressure')
plt.ylabel('affine G\' modulus')
plt.legend()
plt.tight_layout()

#plt.subplots_adjust(top=0.9)
#plt.figtext(0.8,0.25,'\n'.join(regressResult), ha='right', fontsize=12)
#plt.suptitle('Affine Elastic Moduli vs. $\Delta Z$ for non/tri lattice, July 2020, fitting p<{}'.format(cutoffPressure),x=0.5,y=0.93, fontsize = 16)

#plt.savefig('plots/presentation/affine_elasticmoduli-pressure-squ-july2020.png', dpi=300)

# 2. Rattler Plots

## 2.1. Fraction of rattlers vs. $\Delta\phi$
This cell plots each trial separately for more effective comparison, since on the same axes the vast majority of data points would occupy the same region. Titles and axis labels for the graphs have been omitted as these plots were assembled into a single diagram outside of matpl

In [ ]:
plt.figure(figsize=(8,8)) # large, square plot
for trial in data.groupby(independentVariable):
    #trialData=trial[1][trial[1].deltaPhi<=0.15]
    plt.plot(trial[1]['pressure'],trial[1]['rattlerfraction'],'.')
    #rattlerMeans=trialData.groupby('phi')['rattlerfraction'].mean()
    #rattlerStd=trialData.groupby('phi')['rattlerfraction'].std()
    #plt.errorbar(rattlerMeans.index.values, rattlerMeans, yerr=rattlerStd, fmt='.', ecolor=None, label='{}: {}'.format(independentVariable,trial[0]), alpha=0.5)

plt.xlabel('pressure')
plt.ylabel('Fraction of Rattlers')
plt.title('Fraction of Rattlers vs. Pressure for 920 particles, no pins')
#plt.legend()
plt.savefig('plots/non0-920-rattlers_vs_pressure.png', dpi=300)
plt.show()